# 优化模型参数

现在我们有了一个模型和数据，是时候通过优化其参数来训练、验证和测试我们的模型了。训练模型是一个迭代的过程；在每次迭代中，模型对输出进行预测，计算其预测的误差（*损失*），收集误差相对于其参数的导数，然后使用梯度下降**优化**这些参数。

## 前置代码
我们先加载前几节关于Dataset, DataLoader和构建模型的代码。

In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer


class SST2Dataset(Dataset):
    def __init__(self, file_path, split):
        file_path = f"{file_path}/{split}.jsonl"
        self.datas = pd.read_json(file_path, lines=True).to_dict(orient='records')

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        return self.datas[idx]

max_length = 128  # 最大文本长度

def my_collate_fn(batch):
    # 提取每个样本中的文本和标签信息
    texts = [sample['text'] for sample in batch]
    labels = [sample['label'] for sample in batch]

    # 截断或填充文本以确保不超过最大长度
    texts_list = []
    for text in texts:
        encoded_text = tokenizer.encode(text, truncation=True, max_length=max_length, padding='max_length')
        texts_list.append(encoded_text)

    texts_tensor = [torch.Tensor(text) for text in texts_list]

    # 将文本序列填充为相同长度
    texts_tensor = pad_sequence(texts_tensor, batch_first=True, padding_value=0)

    # 将标签转换为张量
    labels_tensor = torch.tensor(labels)

    return {'text': texts_tensor, 'label': labels_tensor}

training_data = SST2Dataset(".", "train")
test_data = SST2Dataset(".", "test")

# Load bert_tokenizer locally
tokenizer = AutoTokenizer.from_pretrained("./")

train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True, collate_fn=my_collate_fn)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True, collate_fn=my_collate_fn)



class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(max_length, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 超参数

超参数是可调参数，允许我们控制模型优化过程。
不同的超参数值可以影响模型的训练和收敛速度

我们为训练定义了以下超参数：
 - **Epoch的数量** - 迭代数据集的次数
 - **批量大小** - 在更新参数之前通过网络传播的数据样本数量
 - **学习率** - 在每个批次/迭代次数中更新模型参数的程度。较小的值会导致较慢的学习速度，而较大的值可能会导致训练过程中出现不可预测的行为。

In [2]:
learning_rate = 1e-3
batch_size = 8
epochs = 5

## 优化循环

一旦设置了超参数，就可以使用优化循环来训练和优化模型。优化循环的每个迭代称为一个**epoch**。

每个epoch包括两个主要部分：
 - **训练循环** - 遍历训练数据集并尝试收敛到最佳参数。
 - **验证/测试循环** - 遍历测试数据集以检查模型性能是否在改善。

### 损失函数

当面对一些训练数据时，我们未经训练的网络很可能不会给出正确的答案。**损失函数**用于测量获得的结果与目标值之间的不相似程度，而我们希望在训练过程中将损失函数最小化。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实的数据标签值进行比较。

常见的损失函数包括用于回归任务的[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)（均方误差）和用于分类任务的[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)（负对数似然）。[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)结合了``nn.LogSoftmax``和``nn.NLLLoss``。

我们将模型的输出logits传递给``nn.CrossEntropyLoss``，它将标准化logits并计算预测误差。


In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### 优化器

优化是调整模型参数以减小每个训练步骤中模型误差的过程。**优化算法**定义了如何执行此过程（在本例中，我们使用随机梯度下降）。
所有优化逻辑都封装在``optimizer``对象中。在这里，我们使用了SGD优化器；此外，PyTorch还提供了许多[不同的优化器](https://pytorch.org/docs/stable/optim.html)，如ADAM和RMSProp，它们对于不同类型的模型和数据效果更好。

我们通过传入需要训练的模型参数并输入学习率超参数来初始化优化器。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环内部，优化过程分为三个步骤：
 * 调用``optimizer.zero_grad()``来重置模型参数的梯度。梯度默认会累积；为了防止重复计算，我们在每次迭代中显式将它们归零。
 * 通过调用``loss.backward()``来反向传播预测损失。PyTorch会将损失相对于每个参数的梯度存储起来。
 * 一旦我们有了梯度，就调用``optimizer.step()``来通过在反向传播中收集的梯度来调整参数。


## 完整实现
我们定义了``train_loop``，它循环执行我们的优化代码，以及``test_loop``，它评估模型在测试数据上的性能。

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    # 设置模型为训练模式 - 对于批量归一化和dropout层很重要
    # 在这种情况下不是必需的，但为了最佳实践添加了这个步骤。
    model.train()
    for batch, inputs in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(inputs["text"])
        loss = loss_fn(pred, inputs["label"])

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(inputs["label"])
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 设置模型为评估模式 - 对于批量归一化和dropout层很重要
    # 在这种情况下不是必需的，但为了最佳实践添加了这个步骤。
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # 使用torch.no_grad()评估模型可以确保在测试模式下不计算梯度
    # 还有助于减少需要计算梯度的张量的不必要的梯度计算和内存使用。
    with torch.no_grad():
        for inputs in dataloader:
            pred = model(inputs["text"])
            test_loss += loss_fn(pred, inputs["label"]).item()
            correct += (pred.argmax(1) == inputs["label"]).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化损失函数和优化器，并将它们传递给``train_loop``和``test_loop``。
随时增加epoch的数值以跟踪模型性能的改进。

（本教程只将流程跑通，因此虽然loss下降但效果并不一定会提升。）

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 79.145821  [   16/ 6920]
loss: 8.649393  [ 1616/ 6920]
loss: 2.784671  [ 3216/ 6920]
loss: 1.026397  [ 4816/ 6920]
loss: 0.612357  [ 6416/ 6920]
Test Error: 
 Accuracy: 52.1%, Avg loss: 0.763575 

Epoch 2
-------------------------------
loss: 0.613927  [   16/ 6920]
loss: 0.660455  [ 1616/ 6920]
loss: 0.635091  [ 3216/ 6920]
loss: 1.044611  [ 4816/ 6920]
loss: 0.670879  [ 6416/ 6920]
Test Error: 
 Accuracy: 50.6%, Avg loss: 0.762069 

Epoch 3
-------------------------------
loss: 0.707522  [   16/ 6920]
loss: 0.591276  [ 1616/ 6920]
loss: 0.855261  [ 3216/ 6920]
loss: 0.649688  [ 4816/ 6920]
loss: 0.727412  [ 6416/ 6920]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.717605 

Epoch 4
-------------------------------
loss: 0.649030  [   16/ 6920]
loss: 0.659693  [ 1616/ 6920]
loss: 0.703008  [ 3216/ 6920]
loss: 0.662853  [ 4816/ 6920]
loss: 0.690132  [ 6416/ 6920]
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.725939 

Epoch 5
-----------------------